In [ ]:
# Importar bibliotecas

import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Parâmetros:

N =  60#@param

U = 0.17#@param

J1 = 1/np.sqrt(2) #@param
J3 = 1/np.sqrt(2) #@param

e =  0#@param

# Dimensão da matriz

D = int(((N+2)*(N+1))/2)

# função delta

def delta(n,m):
  if n == m:
    return 1
  else:
    return 0

# Estado inicial

aux = np.array([0 for i in range(D)])
aux[D-1] = 1
estado_inicial = aux.reshape(D, 1)

In [ ]:
# Hamiltoniano 

def H(n1, n2, n3, m1, m2, m3):
  f1 = delta(n1,m1)*delta(n2,m2)*delta(n3,m3)*U*(n1-n2+n3)**2
  f2 = J1*(delta(n1 + 1,m1)*delta(n2 - 1,m2)*delta(n3,m3)*np.sqrt(n2*(n1 + 1)) + delta(n1 - 1,m1)*delta(n2 + 1,m2)*delta(n3,m3)*np.sqrt(n1*(n2 + 1)))
  f3 = J3*(delta(n1,m1)*delta(n2 + 1,m2)*delta(n3 - 1,m3)*np.sqrt(n3*(n2 + 1)) + delta(n1,m1)*delta(n2 - 1,m2)*delta(n3 + 1,m3)*np.sqrt(n2*(n3 + 1)))
  f4 = e*(n3-n1)*delta(n1,m1)*delta(n3,m3)
  return f1 + f2 + f3 + f4  

In [ ]:
# Matrizes

# Matriz Hamiltoniano

def Matriz_H(N):
  matriz = np.zeros((D,D))
  l = 0
  c = 0
  for n1 in range(N+1):
    for n2 in range(N+1-n1):
      c = 0
      for m1 in range(N+1):
        for m2 in range(N+1-m1):
          matriz[l][c] = H(m1,m2,N-m1-m2,n1,n2,N-n1-n2)
          c += 1
      l += 1
  return matriz

H = Matriz_H(N)

# N1

def N1(N):
  N1 = np.zeros( (D,D) )
  l = 0
  c = 0
  for n1 in range(N+1):
    for n2 in range(N+1-n1):
      c = 0
      for m1 in range(N+1):
        for m2 in range(N+1-m1):
          N1[l][c] = m1*delta(n1,m1)*delta(n2,m2)
          c += 1
      l += 1
  return N1

# N2

def N2(N):
  N2 = np.zeros( (D,D) )
  l = 0
  c = 0
  for n1 in range(N+1):
    for n2 in range(N+1-n1):
      c = 0
      for m1 in range(N+1):
        for m2 in range(N+1-m1):
          N2[l][c] = m2*delta(n1,m1)*delta(n2,m2)
          c += 1
      l += 1
  return N2

# N3

def N3(N):
  N3 = np.zeros( (D,D) )
  l = 0
  c = 0
  for n1 in range(N+1):
    for n2 in range(N+1-n1):
      c = 0
      for m1 in range(N+1):
        for m2 in range(N+1-m1):
          N3[l][c] = (N-m1-m2)*delta(n1,m1)*delta(n2,m2)
          c += 1
      l += 1
  return N3

In [ ]:
# Extrair os autovalores e autovetores

sigma, psi = np.linalg.eigh(H)

In [ ]:
# |psi(t)>

def ket_t(t):
  psi_t = np.zeros((D)).reshape(D,1)
  for i in range(D):
    arg = complex(0, sigma[i]*t)
    bra = psi[:,i].reshape(1,D)
    ket = psi[:,i].reshape(D,1)
    cte = np.exp(arg)*bra.dot(estado_inicial)[0][0]
    psi_t = psi_t + cte*ket
  return psi_t

#<psi(t)|

def bra_t(t):
  ket = ket_t(t)
  bra = np.conj(ket).reshape(1, D)
  return bra

In [ ]:
# Valor esperado

def valor_esperado(t, NU):
  fator1 = bra_t(t).dot(NU(N))
  fator2 = fator1.dot(ket_t(t))
  return fator2[0][0]/N

In [ ]:
# plot

t_total = 500
var = 1000

valor_esperado_array_n1 = np.zeros((var))
valor_esperado_array_n2 = np.zeros((var))
valor_esperado_array_n3 = np.zeros((var))
t_array = np.zeros((var)) 

valor_esperado_array_n1 = np.zeros((var))
for i in range(var):
  valor_esperado_array_n1[i] = valor_esperado_array_n1[i] + valor_esperado((t_total/var)*i, N1)
  valor_esperado_array_n2[i] = valor_esperado_array_n2[i] + valor_esperado((t_total/var)*i, N2)
  valor_esperado_array_n3[i] = valor_esperado_array_n3[i] + valor_esperado((t_total/var)*i, N3)
  t_array[i] = t_array[i] + (t_total/var)*i

In [ ]:
# Plot

plt.figure()


# Gráfico 1

axs1 = plt.subplot(311)
plt.plot(t_array, valor_esperado_array_n1)
plt.ylabel("$<n_1>$")
plt. title("N = 60 - $\\varepsilon = 0.0$ - U = 0.17")
# Gráfico 2

plt.subplot(312, sharex = axs1, sharey = axs1)
plt.plot(t_array, valor_esperado_array_n2, 'r')
plt.ylabel("$<n2>$")

# Gráfico 3

plt.subplot(313, sharex = axs1, sharey = axs1)
plt.plot(t_array, valor_esperado_array_n3, 'c')
plt.ylabel("$<n3>$")
plt.xlabel("t")

# Ajustando a figura

plt.subplots_adjust(top = 1.7,
                    bottom = 0.08,
                    left = 0.10,
                    hspace = 0.0,
                    wspace = 0.5)